In [1]:

import numpy as np
import pandas as pd

In [69]:
path_train = r'C:\Users\Jatin Mittal\Downloads\Compressed\all\train.csv'
path_test = r'C:\Users\Jatin Mittal\Downloads\Compressed\all\test.csv'
df_train = pd.read_csv(path_train)
#X = df.iloc[:,1:]
#Y = df.iloc[:,0]

df_test = pd.read_csv(path_test)
#X = df.iloc[:,1:]
#Y = df.iloc[:,0]

for col in df_train.columns:
    df_train[col] = df_train[col].astype('category')
    
for col2 in df_test.columns:
    df_test[col2] = df_test[col2].astype('category')
    
#df_test=df_test.rename(columns = {'id':'new_name'})


In [70]:
print("Training set")
display(df_train.describe())
print("Testing set")
display(df_test.describe())


Training set


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769,32769,32769,32769
unique,2,7518,4243,128,177,449,343,2358,67,343
top,1,4675,770,117961,118300,117878,118321,117906,290919,118322
freq,30872,839,152,21407,4424,1135,4649,6896,10980,4649


Testing set


,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,912363,912363,912363,912363,912363,912363,912363,912363,912363,912363
unique,912363,3013,3871,121,162,424,362,3463,108,315
top,912363,15716,25262,117961,118327,117945,118321,119831,292795,119082
freq,1,58088,12507,522633,101268,161730,152583,107736,145441,267948


In [4]:
Y = df_train.iloc[:,0]
X_train = df_train.iloc[:,1:]
X_test = df_test.iloc[:,1:]

X_train = X_train.drop(columns=['ROLE_TITLE'])
X_test = X_test.drop(columns=['ROLE_TITLE'])
header = list(df_train.columns[1:])

all_data = X_train.append(X_test)
print(all_data.describe())
print (X_train.describe())

            RESOURCE         MGR_ID  ROLE_ROLLUP_1  ROLE_ROLLUP_2  \
count  945132.000000  945132.000000  945132.000000  945132.000000   
mean    38208.145866   29070.216546  117705.217178  118621.916453   
std     27606.895462   39975.744688    6504.607055    2291.824835   
min         0.000000      25.000000    4292.000000   23779.000000   
25%     18072.000000    5568.000000  117961.000000  118225.000000   
50%     34432.000000   15639.000000  117961.000000  118327.000000   
75%     45324.000000   46634.000000  118256.000000  118446.000000   
max    312153.000000  311823.000000  311178.000000  286791.000000   

       ROLE_DEPTNAME  ROLE_FAMILY_DESC    ROLE_FAMILY     ROLE_CODE  
count  945132.000000     945132.000000  945132.000000  945132.00000  
mean   121465.869494     158866.779366  170923.202926  119247.43254  
std     22733.262100      55403.282656   95266.052334    6132.69639  
min      4674.000000       4673.000000    3130.000000  117880.00000  
25%    118320.000000     119

In [50]:
#trying one-hot encoding on each variable
from sklearn import preprocessing
test_offset=32769
encoder = preprocessing.OneHotEncoder(dtype=np.uint32)
# we want to encode the category IDs encountered both in the training and the test set, so we fit the encoder on both
encoder.fit(all_data)
temp = encoder.transform(all_data)
X = temp[:test_offset,:]
X_test = temp[test_offset:,:]
print(X.shape)
print(X_test.shape)

(32769, 17914)
(912363, 17914)


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,shuffle=True)

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
auc = metrics.auc(fpr,tpr)
print (auc)

In [31]:
#Logistic Regression
from sklearn import linear_model, cross_validation
SEED = 42
mean_auc = 0.0

n = 5  # repeat the CV procedure 5 times to get more precise results
model = linear_model.LogisticRegression(C=1,penalty='l2',class_weight='balanced')
#model = BernoulliNB(alpha=0.005)
#model = ensemble.ExtraTreesClassifier(criterion='entropy',n_jobs=-1,verbose=1)
for i in range(n):
    # for each iteration, randomly hold out 20% of the data as CV set
    X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(X, Y, test_size=.20, random_state=i*SEED)

    # if you want to perform feature selection / hyperparameter
    # optimization, this is where you want to do it

    # train model and make predictions
    model.fit(X_train, y_train) 
    preds = model.predict_proba(X_cv)[:, 1]    
    # compute AUC metric for this CV fold
    fpr, tpr, thresholds = metrics.roc_curve(y_cv, preds)
    roc_auc = metrics.auc(fpr, tpr)
    print(roc_auc)

(6554,)
(6554,)
0.8661299454190626
(6554,)
(6554,)
0.8744457808361431
(6554,)
(6554,)
0.876569042071853
(6554,)
(6554,)
0.8789891880895923
(6554,)
(6554,)
0.85143376392171


In [24]:
preds

array([0.98849423, 0.99904574, 0.97671389, ..., 0.73137782, 0.99693426,
       0.40916996])

In [35]:
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=12)
lg = LogisticRegression(C=1,penalty='l2',class_weight='balanced')      # C=1/alpha here is regularization
lg.fit(x_train,y_train)
y_pred = lg.predict_proba(x_test)[:, 1]
# note: while calculating ROC it is always recommmended to  use predict_prob because in background ROC is calculated using
# probability distribution so always use probab.


LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [41]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred1, pos_label=1)
auc = metrics.auc(fpr,tpr)
print (auc)

0.8781151479064764


In [15]:
#using random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(RandomForestClassifier(class_weight='balanced'))
param = {'randomforestclassifier__n_estimators':[10,30,50,70,100],'randomforestclassifier__max_features':['auto','sqrt'],'randomforestclassifier__criterion':['gini','entropy']}

from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(pipeline,param)



In [18]:
clf.fit(x_train,y_train)
print (clf.best_params_)  												#To view the optimum values of the hyperparameters



{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_features': 'sqrt'}


In [45]:
y_pred = clf.predict_proba(x_test)[:,1]
y_pred.shape

(6554,)

In [46]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
auc = metrics.auc(fpr,tpr)
print (auc)

0.9461554896152168


In [51]:
test_pred = clf.predict_proba(X_test)[:,1]

In [52]:
test_pred.shape

(912363,)

In [54]:
# importing final test data set

path = r'C:\Users\Jatin Mittal\Downloads\Compressed\all\test.csv'
test_data_original = pd.read_csv(path)
test_data = test_data_original['id']
test_data.head()

0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64

In [67]:
#  exporting results to csv file
submit = test_data.copy()
final_ = pd.DataFrame(submit)
final_['ACTION'] = test_pred
final_ = final_[:58921,:]
print(final_.head())
final_.to_csv('amazon_submission2018.csv',index=False)

   id  ACTION
0   1     1.0
1   2     1.0
2   3     1.0
3   4     1.0
4   5     1.0


In [71]:
final_ = final_.iloc[:58921,:]

TypeError: unhashable type: 'slice'